# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 19 2022 9:21AM,244542,19,ELI08222022.1,Eli Lilly and Company,Released
1,Aug 19 2022 9:20AM,244541,12,HH-36471,Hush Hush,Released
2,Aug 19 2022 9:20AM,244541,12,HH-36472,Hush Hush,Released
3,Aug 19 2022 9:20AM,244541,12,HH-36474,Hush Hush,Released
4,Aug 19 2022 9:20AM,244541,12,HH-36475,Hush Hush,Released
5,Aug 19 2022 9:20AM,244541,12,HH-36476,Hush Hush,Released
6,Aug 19 2022 9:20AM,244541,12,HH-36477,Hush Hush,Released
7,Aug 19 2022 9:20AM,244541,12,HH-36478,Hush Hush,Released
8,Aug 19 2022 9:20AM,244541,12,HH-36479,Hush Hush,Released
9,Aug 19 2022 9:10AM,244540,15,8419212,"Mizner Bioscience, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,244538,Released,16
20,244539,Released,1
21,244540,Released,3
22,244541,Released,8
23,244542,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244538,NaN,16.0
244539,NaN,1.0
244540,NaN,3.0
244541,NaN,8.0
244542,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244450,0.0,1.0
244454,0.0,1.0
244455,1.0,0.0
244457,0.0,1.0
244458,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244450,0,1
244454,0,1
244455,1,0
244457,0,1
244458,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244450,0,1
1,244454,0,1
2,244455,1,0
3,244457,0,1
4,244458,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244450,,1
1,244454,,1
2,244455,1,
3,244457,,1
4,244458,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 19 2022 9:21AM,244542,19,Eli Lilly and Company
1,Aug 19 2022 9:20AM,244541,12,Hush Hush
9,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC"
12,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC"
13,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc."
29,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc."
30,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation
45,Aug 19 2022 8:14AM,244532,19,"AdvaGen Pharma, Ltd"
47,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation
54,Aug 19 2022 8:06AM,244528,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 19 2022 9:21AM,244542,19,Eli Lilly and Company,,1
1,Aug 19 2022 9:20AM,244541,12,Hush Hush,,8
2,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC",,3
3,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",,1
4,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",,16
5,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc.",1,
6,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation,,15
7,Aug 19 2022 8:14AM,244532,19,"AdvaGen Pharma, Ltd",2,
8,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation,,7
9,Aug 19 2022 8:06AM,244528,10,ISDIN Corporation,3,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 9:21AM,244542,19,Eli Lilly and Company,1,
1,Aug 19 2022 9:20AM,244541,12,Hush Hush,8,
2,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC",3,
3,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",1,
4,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",16,
5,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc.",,1
6,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation,15,
7,Aug 19 2022 8:14AM,244532,19,"AdvaGen Pharma, Ltd",,2
8,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation,7,
9,Aug 19 2022 8:06AM,244528,10,ISDIN Corporation,10,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 9:21AM,244542,19,Eli Lilly and Company,1,
1,Aug 19 2022 9:20AM,244541,12,Hush Hush,8,
2,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC",3,
3,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",1,
4,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",16,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 9:21AM,244542,19,Eli Lilly and Company,1.0,NaN
1,Aug 19 2022 9:20AM,244541,12,Hush Hush,8.0,NaN
2,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC",3.0,NaN
3,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",1.0,NaN
4,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",16.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 9:21AM,244542,19,Eli Lilly and Company,1.0,0.0
1,Aug 19 2022 9:20AM,244541,12,Hush Hush,8.0,0.0
2,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC",3.0,0.0
3,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",1.0,0.0
4,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",16.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1466958,35.0,3.0
102,244450,1.0,0.0
12,244541,8.0,0.0
15,733617,20.0,0.0
16,978047,5.0,1.0
17,244455,0.0,1.0
19,1222621,3.0,3.0
20,244459,17.0,0.0
22,244537,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1466958,35.0,3.0
1,102,244450,1.0,0.0
2,12,244541,8.0,0.0
3,15,733617,20.0,0.0
4,16,978047,5.0,1.0
5,17,244455,0.0,1.0
6,19,1222621,3.0,3.0
7,20,244459,17.0,0.0
8,22,244537,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,35.0,3.0
1,102,1.0,0.0
2,12,8.0,0.0
3,15,20.0,0.0
4,16,5.0,1.0
5,17,0.0,1.0
6,19,3.0,3.0
7,20,17.0,0.0
8,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,102,Released,1.0
2,12,Released,8.0
3,15,Released,20.0
4,16,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,22
Status,,,,,,,,,
Executing,3.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0
Released,35.0,1.0,8.0,20.0,5.0,0.0,3.0,17.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,22
0,Executing,3.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0
1,Released,35.0,1.0,8.0,20.0,5.0,0.0,3.0,17.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,22
0,Executing,3.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0
1,Released,35.0,1.0,8.0,20.0,5.0,0.0,3.0,17.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()